In [1]:
#7
#athena db config - llmprice
#AAI-540 Group 3 FP

In [2]:
import boto3
import sagemaker
from pyathena import connect
import pandas as pd
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
ingest_create_athena_db_passed = False

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [3]:
database_name = "dsoaws"
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [4]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)
pd.read_sql(statement, conn)

CREATE DATABASE IF NOT EXISTS dsoaws


/tmp/ipykernel_8916/2532319348.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [5]:
statement = "SHOW DATABASES"
df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_8916/1677879827.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,database_name
0,default
1,dsoaws


In [6]:
s3_data_path = 's3://sagemaker-us-east-1-342942554664/llmpricingdata.csv'
s3_table_path = 's3://sagemaker-us-east-1-342942554664/table4/'

In [7]:
!aws s3 cp {s3_data_path} {s3_table_path}

copy: s3://sagemaker-us-east-1-342942554664/llmpricingdata.csv to s3://sagemaker-us-east-1-342942554664/table4/llmpricingdata.csv


In [8]:
#table4 - llmpricing
table_name_csv = 'llmpricing'
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)
ingest_create_athena_table_csv_passed = False
dataexplore = pd.read_csv('llmpricingdata.csv')
dataexplore.head()

,provider,developer,model,input_token_1k_usd,output_token_1k_usd,context_size
0,OpenAI,OpenAI,gpt-4,0.030,0.060,8000
1,OpenAI,OpenAI,gpt-4-32k,0.060,0.120,32000
2,OpenAI,OpenAI,gpt-4-1106-preview,0.010,0.030,128000
3,OpenAI,OpenAI,gpt-4-1106-vision-preview,0.010,0.030,128000
4,OpenAI,OpenAI,gpt-3.5-turbo-1106,0.001,0.002,16000


In [9]:
dataexplore.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   provider             31 non-null     object 
 1   developer            31 non-null     object 
 2   model                31 non-null     object 
 3   input_token_1k_usd   31 non-null     float64
 4   output_token_1k_usd  31 non-null     float64
 5   context_size         31 non-null     int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 1.6+ KB


In [10]:
!aws s3 cp s3://sagemaker-us-east-1-342942554664/llmpricingdata.csv \
           s3://sagemaker-us-east-1-342942554664/table4/

copy: s3://sagemaker-us-east-1-342942554664/llmpricingdata.csv to s3://sagemaker-us-east-1-342942554664/table4/llmpricingdata.csv


In [11]:
!aws s3 ls s3://sagemaker-us-east-1-342942554664/table4/

2026-02-12 09:55:24       1587 llmpricingdata.csv


In [12]:
#sql table creation
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         provider STRING,
         developer STRING,
         model STRING,
         input_token_1k_usd FLOAT,
         output_token_1k_usd FLOAT,
         context_size INT
) ROW FORMAT DELIMITED 
  FIELDS TERMINATED BY ',' 
  LINES TERMINATED BY '\\n' 
LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table_name_csv, s3_table_path
)

In [13]:
pd.read_sql(statement, conn)

/tmp/ipykernel_8916/618921652.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [14]:
statement = "SHOW TABLES in {}".format(database_name)
df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_8916/84333453.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,tab_name
0,aimodelpoll
1,lifearchitect
2,lifearchitectmodels
3,llmachievements
4,llmpricing


In [15]:
statement = """SELECT * FROM {}.{} LIMIT 10""".format(
    database_name, table_name_csv
)
print(statement)
df = pd.read_sql(statement, conn)
df.head()

SELECT * FROM dsoaws.llmpricing LIMIT 10


/tmp/ipykernel_8916/888292236.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


,provider,developer,model,input_token_1k_usd,output_token_1k_usd,context_size
0,OpenAI,OpenAI,gpt-4,0.030,0.060,8000
1,OpenAI,OpenAI,gpt-4-32k,0.060,0.120,32000
2,OpenAI,OpenAI,gpt-4-1106-preview,0.010,0.030,128000
3,OpenAI,OpenAI,gpt-4-1106-vision-preview,0.010,0.030,128000
4,OpenAI,OpenAI,gpt-3.5-turbo-1106,0.001,0.002,16000
